# Basic QGAN

Quantum Generative Adversarial Network capable of generating a fixed state of a single qubit. Implemented following https://arxiv.org/pdf/1804.08641.pdf by Cameron Estabrooks, Jacob Ewaniuk, Adam Grace, Spencer Hill, and Joshua Otten.


# Imports

In [16]:
import numpy as np
from numpy import pi
import qiskit
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.algorithms.optimizers import SPSA
import scipy
from numpy import linalg as la
from qiskit.quantum_info import Statevector
from tensorflow import keras
np.random.seed(0)

# Real Data

In [25]:
def real(qc, angles):
    assert len(angles == 3)
    qc.h(0)
    qc.u(angles[0], angles[1], angles[2], 0)

# Generator Function

Meant to create a universal ansatz capable of mimicking any single-qubit gate and entanglement. Weights are available for optimization. 

In [3]:
def generator(weights, circ):
    circ.h(0)
    circ.rx(weights[0], 0)
    circ.rx(weights[1], 1)
    circ.ry(weights[2], 0)
    circ.ry(weights[3], 1)
    circ.rz(weights[4], 0)
    circ.rz(weights[5], 1)
    circ.cnot(0, 1)
    circ.rx(weights[6], 0)
    circ.ry(weights[7], 0)
    circ.rz(weights[8], 0)
    

# Discriminator Function

In [4]:
def discriminator(w, circuit):
    circuit.h(0)
    circuit.rx(w[0], 0)
    circuit.rx(w[1], 2)
    circuit.ry(w[2], 0)
    circuit.ry(w[3], 2)
    circuit.rz(w[4], 0)
    circuit.rz(w[5], 2)
    circuit.cx(0, 2)
    circuit.rx(w[6], 2)
    circuit.ry(w[7], 2)
    circuit.rz(w[8], 2)
    circuit.measure(2,0)

# Cost Functions

In [5]:
# Circuit functions for real and fake data
def real_disc_circuit(angles, local_disc_weights):
    qc = QuantumCircuit(3, 1)
    real(qc, angles)
    discriminator(local_disc_weights, qc)
    return qc

def gen_disc_circuit(local_gen_weights, local_disc_weights):
    qc = QuantumCircuit(3, 1)
    generator(local_gen_weights, qc)
    discriminator(local_disc_weights, qc)
    return qc

In [6]:
def disc_cost(local_disc_weights, shots=1024, sim=Aer.get_backend('aer_simulator')):
    qc = real_disc_circuit(angs, local_disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    memory = sim.run(qc.compose(circ_trans), shots=shots, memory=True).result().get_memory(qc)
    ones = memory.count('1')
    zeros = memory.count('0')
    prob_disc = ones/(zeros+ones)
    
    qc = gen_disc_circuit(gen_weights, disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    memory = sim.run(qc.compose(circ_trans), shots=shots, memory=True).result().get_memory(qc)
    ones = memory.count('1')
    zeros = memory.count('0')
    prob_gen = ones/(zeros+ones)
    return prob_gen - prob_disc

def gen_cost(local_gen_weights, shots=1024, sim=Aer.get_backend('aer_simulator')):
    qc = gen_disc_circuit(local_gen_weights, disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    memory = sim.run(qc.compose(circ_trans), shots=shots, memory=True).result().get_memory(qc)
    ones = memory.count('1')
    zeros = memory.count('0')
    prob_gen = ones/(zeros+ones)
    return -prob_gen

# QGAN Training

In [12]:
# Discriminator Training Routine/Function
def disc_train(disc_cost_function, local_disc_weights, hp=None):
    """
    Trains the QGAN discriminator
    :param disc_cost_fun: Cost function for discriminator
    :param disc_weights: Weights/parameters for discriminator
    :param hp: A dictionary of hyperparameters: "steps_per_epoch", "learn_rate"
    :return:
    """
    
    if hp is None:
        hp = {"steps_per_epoch": 50, "learn_rate": 0.001}
    for step in range(hp["steps_per_epoch"]):
        if step % 2 == 0:
            print("Step: ", step, "Disc Loss: ", disc_cost(local_disc_weights))
        spsa = SPSA(maxiter=300)
        result = spsa.optimize(len(local_disc_weights), disc_cost_function, initial_point=local_disc_weights)
        local_disc_weights = result[0]
    return local_disc_weights

In [13]:
# Generator Training Routine/Fuction
def gen_train(gen_cost_function, local_gen_weights, hp=None):
    """
    Trains the QGAN discriminator
    :param gen_cost_fun: Cost function for generator
    :param gen_weights: Weights/parameters for generator
    :param hp: A dictionary of hyperparameters: "steps_per_epoch", "learn_rate"
    :return:
    """
    
    if hp is None:
        hp = {"steps_per_epoch": 10, "learn_rate": 0.001}
    for step in range(hp["steps_per_epoch"]):
        if step % 2 == 0:
            print("Step: ", step, "Gen Loss: ", gen_cost(local_gen_weights))
        spsa = SPSA(maxiter=300)
        result = spsa.optimize(len(local_gen_weights), gen_cost_function, initial_point=local_gen_weights)
        local_gen_weights = result[0]
    return local_gen_weights
#     opt = keras.optimizers.Nadam(learning_rate=hp["learn_rate"])
#     for step in range(hp["steps_per_epoch"]):
#         opt.minimize(gen_cost_function, gen_weights)

In [9]:
# Full Training Heuristics
def train(disc_cost_fun, gen_cost_fun, hp=None):
    """
    Trains the QGAN
    :param disc_cost_fun: Cost function for discriminator
    :param gen_cost_fun: Cost function for generator
    :param disc_weights: Weights/parameters for discriminator
    :param gen_weights: Weights/parameters for generator
    :param hp: A dictionary of hyperparameters: "epochs", "steps_per_epoch", "learn_rate"
    :return:
    """
    global gen_weights
    global disc_weights
    if hp is None:
        hp = {"epochs": 1, "steps_per_epoch": 10, "learn_rate": 0.001}
    for epoch in range(hp["epochs"]):
        disc_weights = disc_train(disc_cost_fun, disc_weights, hp)
        gen_weights = gen_train(gen_cost_fun, gen_weights, hp)

# Testing

In [23]:
def compare(real_circuit, generator_circuit):
    global angs
    global gen_weights
    qc_real = QuantumCircuit(1)
    real_circuit(qc_real, angs)
    qc_fake = QuantumCircuit(3,1)
    generator_circuit(gen_weights, qc_fake)
    return Statevector.from_instruction(qc_real), Statevector.from_instruction(qc_fake)

In [14]:
# Initialize random fixed state
disc_weights = np.random.normal(size=(9,))
gen_weights = np.array([np.pi] + [0] * 8) + \
               np.random.normal(scale=1e-2, size=(9,))
angs = np.random.normal(size=(3,))

# Train the disc and generator 
train(disc_cost, gen_cost)

# Print trained weights
print("Trained Weights: ")
print("Discriminator: ", disc_weights)
print("Generator: ", gen_weights)

Original Weights: 
Discriminator:  [ 0.6536186   0.8644362  -0.74216502  2.26975462 -1.45436567  0.04575852
 -0.18718385  1.53277921  1.46935877]
Generator:  [ 3.14314213e+00  3.78162520e-03 -8.87785748e-03 -1.98079647e-02
 -3.47912149e-03  1.56348969e-03  1.23029068e-02  1.20237985e-02
 -3.87326817e-03]
Step:  0 Disc Loss:  -0.0126953125
Step:  2 Disc Loss:  -0.4111328125
Step:  4 Disc Loss:  -0.4345703125
Step:  6 Disc Loss:  -0.431640625
Step:  8 Disc Loss:  -0.4189453125
Step:  0 Gen Loss:  -0.99609375
Step:  2 Gen Loss:  -0.9990234375
Step:  4 Gen Loss:  -1.0
Step:  6 Gen Loss:  -0.9990234375
Step:  8 Gen Loss:  -0.9990234375
Trained Weights: 
Discriminator:  [ -3.41809413 -14.1323331  -28.48113143  -0.12317608 -16.31772143
  17.35705172   4.33076467  14.07201074   6.93941813]
Generator:  [ -0.23655427  -6.75047505  14.2364156  -14.03488057   3.13190102
  21.0275211    1.90615101 -16.12890305   5.12380073]


In [36]:
real_state, fake_state = compare(real, generator)

print("Real Statevector: ", real_state._data)
print("Generated Statevector: ", fake_state._data[:2])

Real Statevector:  [ 0.71503777-0.10526565j -0.59972387-0.34346966j]
Generated Statevector:  [-0.41425477-0.08738008j -0.45068986-0.26884937j]
